# 🎯 Phase 2: Decision Transformer 핵심 개념

> **목표**: Decision Transformer의 **핵심 아이디어**인 Return-to-Go(RTG)와 시퀀스 구성 방식을 이해합니다.

---

## 📚 이 노트북에서 배우는 것

| 섹션 | 핵심 개념 | 왜 중요한가? |
|:---:|:---|:---|
| **1** | Return-to-Go (RTG) | DT의 "목표 설정" 방법 |
| **2** | 시퀀스 구성 방식 | 입력 데이터 형태 이해 |
| **3** | 기존 RL과의 차이점 | DT의 독특한 접근법 이해 |
| **4** | 직접 구현해보기 | 코드로 확인 |

---

## 🌟 Decision Transformer의 핵심 아이디어

```
┌─────────────────────────────────────────────────────────────────────┐
│              Decision Transformer의 혁신적 발상                     │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   🤔 기존 RL의 질문:                                                │
│      "이 상태에서 최적의 행동은 뭐지?"                              │
│      → Value Function을 추정해야 함 (어렵고 불안정)                │
│                                                                     │
│   💡 Decision Transformer의 질문:                                   │
│      "이 상태에서 100점을 얻으려면 뭘 해야 하지?"                  │
│      → 목표(RTG)를 조건으로 행동 예측 (Supervised Learning!)       │
│                                                                     │
│   ┌────────────────────────────────────────────────────────┐        │
│   │                                                        │        │
│   │    입력: [목표(RTG), 상태, 과거행동들...]              │        │
│   │                     ↓                                  │        │
│   │              Transformer                               │        │
│   │                     ↓                                  │        │
│   │    출력: 다음 행동(Action)                             │        │
│   │                                                        │        │
│   └────────────────────────────────────────────────────────┘        │
│                                                                     │
│   핵심: "높은 점수를 원하면 높은 RTG를 입력하면 됨!"                │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 📋 목차

1. [🎯 Return-to-Go (RTG) 개념](#1-return-to-go-rtg-개념)
   - RTG란 무엇인가?
   - RTG vs Return
   - 학습 vs 추론에서의 RTG
2. [📦 시퀀스 구성 방식](#2-시퀀스-구성-방식)
   - (RTG, State, Action) 트리플
   - 인터리빙 구조
   - 예측 위치 (State에서 Action 예측)
3. [🔄 기존 RL과의 차이점](#3-기존-rl과의-차이점)
   - Bellman Equation vs Sequence Modeling
   - 장단점 비교
4. [🛠️ 직접 구현해보기](#4-직접-구현해보기)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 재현성을 위한 시드 설정
np.random.seed(42)
torch.manual_seed(42)

# 한글 폰트 설정
def setup_korean_font():
    """시스템에서 사용 가능한 한글 폰트를 찾아 설정"""
    korean_fonts = [
        'Noto Sans CJK KR', 'NanumSquare', 'NanumGothic',
        'Malgun Gothic', 'AppleGothic'
    ]
    system_fonts = {f.name for f in fm.fontManager.ttflist}
    
    for font in korean_fonts:
        if font in system_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    
    # 부분 매칭 시도
    for font in korean_fonts:
        for sys_font in system_fonts:
            if font.lower().replace(' ', '') in sys_font.lower().replace(' ', ''):
                plt.rcParams['font.family'] = sys_font
                plt.rcParams['axes.unicode_minus'] = False
                return sys_font
    
    plt.rcParams['axes.unicode_minus'] = False
    return None

setup_korean_font()

---

# 🎯 1. Return-to-Go (RTG) 개념

RTG는 Decision Transformer의 **가장 중요한 개념**입니다!

---

## 📖 RTG란 무엇인가?

**Return-to-Go (RTG)**는 **"현재 시점부터 에피소드 끝까지 얻을 보상의 합"**입니다.

$$\hat{R}_t = \sum_{t'=t}^{T} r_{t'} = r_t + r_{t+1} + r_{t+2} + \cdots + r_T$$

```
┌─────────────────────────────────────────────────────────────────────┐
│                       RTG 직관적 이해                               │
├─────────────────────────────────────────────────────────────────────┤
│                                                                     │
│   시간:     t=0      t=1      t=2      t=3      t=4    (끝)        │
│   보상:     +1       +2       +3       +4       +5                  │
│                                                                     │
│   RTG[0] = 1+2+3+4+5 = 15  "앞으로 15점을 얻을 것"                 │
│   RTG[1] =   2+3+4+5 = 14  "앞으로 14점을 얻을 것"                 │
│   RTG[2] =     3+4+5 = 12  "앞으로 12점을 얻을 것"                 │
│   RTG[3] =       4+5 = 9   "앞으로 9점을 얻을 것"                  │
│   RTG[4] =         5 = 5   "앞으로 5점을 얻을 것"                  │
│                                                                     │
│   📉 RTG는 시간이 지날수록 감소합니다!                             │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

---

## 🎮 RTG의 역할

> **핵심 아이디어**: RTG를 "목표 점수"로 사용합니다!

```
┌─────────────────────────────────────────────────────────────────────┐
│                                                                     │
│   "RTG=100, 현재 상태=s" → 모델 → "100점을 위한 행동=a"           │
│                                                                     │
│   높은 RTG 입력 → 좋은 행동 출력                                   │
│   낮은 RTG 입력 → 평범한 행동 출력                                 │
│                                                                     │
└─────────────────────────────────────────────────────────────────────┘
```

아래 코드에서 RTG 계산을 직접 해봅시다! 👇

In [ ]:
# RTG 계산 과정을 단계별로 보여주기

def calculate_rtg_verbose(rewards):
    """
    Return-to-Go 계산 (단계별 출력)
    
    핵심: 뒤에서부터 누적합을 계산합니다.
    왜? RTG[t] = reward[t] + reward[t+1] + ... + reward[T]
        즉, "현재부터 끝까지의 합"
    """
    n = len(rewards)
    rtg = [0] * n
    
    print("RTG 계산 과정 (뒤에서부터):")
    print("=" * 60)
    
    cumulative = 0
    for t in range(n - 1, -1, -1):  # 뒤에서부터 순회
        cumulative += rewards[t]
        rtg[t] = cumulative
        
        # 계산 과정 출력
        remaining = rewards[t:]
        formula = " + ".join(map(str, remaining))
        print(f"t={t}: RTG[{t}] = {formula} = {cumulative}")
    
    return rtg

# 예시
rewards = [1, 2, 3, 4, 5]
print(f"보상 시퀀스: {rewards}\n")
rtg = calculate_rtg_verbose(rewards)
print(f"\n최종 RTG: {rtg}")

In [ ]:
# RTG vs Return 비교
# Return: 에피소드 전체의 총 보상 (하나의 숫자)
# RTG: 각 시점에서 남은 보상의 합 (시점마다 다른 값)

rewards = [1, 2, 3, 4, 5]

# Return (전체 합)
total_return = sum(rewards)
print(f"Return (전체 보상 합): {total_return}")
print()

# RTG (각 시점에서의 남은 보상)
print("RTG (각 시점에서 남은 보상의 합):")
print("-" * 40)
for t in range(len(rewards)):
    remaining_rewards = rewards[t:]
    rtg_t = sum(remaining_rewards)
    print(f"t={t}: 남은 보상 = {remaining_rewards} → RTG = {rtg_t}")

print()
print("┌─────────────────────────────────────────────────┐")
print("│  핵심 차이:                                      │")
print("│  - Return: 하나의 숫자 (에피소드 끝에 확정)      │")
print("│  - RTG: 시점마다 다른 값 (점점 감소)             │")
print("│                                                  │")
print("│  RTG는 '아직 얻어야 할 보상'을 의미!             │")
print("└─────────────────────────────────────────────────┘")

### RTG의 역할: 학습 vs 추론

| 단계 | RTG의 역할 | 값의 출처 |
|-----|----------|---------|
| **학습** | 데이터에서 계산된 실제 RTG 사용 | 과거 데이터 |
| **추론** | 원하는 목표 return을 RTG로 설정 | 사용자 지정 |

```
┌─────────────────────────────────────────────────────────────┐
│  학습 시:                                                   │
│    과거 에피소드에서 실제로 얻었던 RTG 사용                 │
│    "이 상태에서 RTG=50이었을 때 실제로 한 행동은 뭐였지?"  │
│                                                             │
│  추론 시:                                                   │
│    원하는 목표를 RTG로 설정                                 │
│    "이 상태에서 RTG=100을 얻으려면 뭘 해야 해?"            │
│    → 높은 RTG를 설정하면 더 좋은 행동을 출력!              │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# 추론 시 RTG 업데이트 과정을 단계별로 시뮬레이션

def simulate_inference_detailed(target_return, actual_rewards):
    """
    추론 시 RTG가 어떻게 변하는지 상세 시뮬레이션
    
    핵심 규칙: 새 RTG = 이전 RTG - 받은 보상
    
    Args:
        target_return: 처음에 설정한 목표 return
        actual_rewards: 실제로 받은 보상들
    """
    print("=" * 60)
    print(f"목표 Return: {target_return}")
    print("=" * 60)
    print()
    
    rtg = target_return
    rtg_history = [rtg]
    
    for step, reward in enumerate(actual_rewards):
        print(f"┌─ Step {step} ─────────────────────────────────────────┐")
        print(f"│  현재 RTG: {rtg}")
        print(f"│  → 모델에게 'RTG={rtg}을 달성하는 행동' 요청")
        print(f"│  → 모델이 행동 출력")
        print(f"│  → 환경에서 행동 실행")
        print(f"│  → 받은 보상: {reward}")
        print(f"│")
        
        new_rtg = rtg - reward
        print(f"│  RTG 업데이트: {rtg} - {reward} = {new_rtg}")
        print(f"│  ('{reward}점 얻었으니, 앞으로 {new_rtg}점 더 필요')")
        print(f"└────────────────────────────────────────────────────┘")
        print()
        
        rtg = new_rtg
        rtg_history.append(rtg)
    
    total_reward = sum(actual_rewards)
    print("=" * 60)
    print(f"최종 결과:")
    print(f"  - 목표: {target_return}")
    print(f"  - 실제 획득: {total_reward}")
    print(f"  - 남은 RTG: {rtg} (0이면 목표 달성!)")
    print(f"  - 달성률: {total_reward/target_return*100:.1f}%")
    
    return rtg_history

# 예시: 목표 100점
target = 100
rewards = [20, 15, 25, 10, 30]
rtg_history = simulate_inference_detailed(target, rewards)

In [ ]:
# RTG 변화 시각화
plt.figure(figsize=(10, 5))

steps = range(len(rtg_history))
plt.plot(steps, rtg_history, 'o-', linewidth=2, markersize=8, color='coral')
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)

# 각 포인트에 값 표시
for i, rtg in enumerate(rtg_history):
    plt.annotate(f'{rtg:.0f}', (i, rtg), textcoords="offset points", 
                 xytext=(0, 10), ha='center')

plt.xlabel('Step')
plt.ylabel('Return-to-Go')
plt.title('추론 시 RTG 변화 (목표: 100)')
plt.grid(True, alpha=0.3)
plt.show()

print("핵심: 매 스텝마다 RTG = RTG - reward 로 업데이트")
print("      RTG는 '아직 얻어야 할 보상'을 의미합니다!")

### 2.1 시퀀스 구조

Decision Transformer는 (RTG, State, Action) 트리플을 시퀀스로 구성합니다.

```
τ = (R̂₁, s₁, a₁, R̂₂, s₂, a₂, ..., R̂ₜ, sₜ, aₜ)
```

- R̂: Return-to-Go ("앞으로 얻고 싶은 보상")
- s: State (현재 상태)
- a: Action (취한 행동)

**왜 이 순서인가?**

```
GPT의 다음 토큰 예측 방식과 연결:

"RTG와 State를 보면 → Action을 예측할 수 있다"

입력:  [R₀, s₀]  →  출력: a₀
입력:  [R₀, s₀, a₀, R₁, s₁]  →  출력: a₁
...

즉, "목표(RTG) + 현재 상태(s) → 행동(a)" 라는 인과 관계!
```

In [ ]:
# 시퀀스 구성 과정을 단계별로 보여주기

def calculate_rtg(rewards):
    """RTG 계산 (간단 버전)"""
    rtg = []
    cumulative = 0
    for r in reversed(rewards):
        cumulative += r
        rtg.append(cumulative)
    return list(reversed(rtg))

def create_dt_sequence_verbose(states, actions, rewards):
    """
    Decision Transformer 시퀀스 생성 (상세 버전)
    
    입력: states, actions, rewards (각각 리스트)
    출력: [RTG₀, s₀, a₀, RTG₁, s₁, a₁, ...] 형태의 시퀀스
    """
    # 1. RTG 계산
    rtgs = calculate_rtg(rewards)
    
    print("Step 1: RTG 계산")
    print("-" * 50)
    for t, (r, rtg) in enumerate(zip(rewards, rtgs)):
        print(f"  t={t}: reward={r}, RTG={rtg}")
    print()
    
    # 2. 시퀀스 구성
    print("Step 2: 시퀀스 구성 (인터리빙)")
    print("-" * 50)
    
    sequence = []
    for t, (rtg, state, action) in enumerate(zip(rtgs, states, actions)):
        # RTG, State, Action 순서로 추가
        sequence.append(('RTG', rtg))
        sequence.append(('State', state))
        sequence.append(('Action', action))
        
        print(f"  t={t}: RTG={rtg} → State={state} → Action={action}")
    
    print()
    print("Step 3: 최종 시퀀스")
    print("-" * 50)
    for i, (token_type, value) in enumerate(sequence):
        print(f"  Position {i:2d}: [{token_type:6s}] = {value}")
    
    return sequence

# 예시 데이터
states = ['s0', 's1', 's2', 's3']
actions = ['a0', 'a1', 'a2', 'a3']
rewards = [1, 2, 3, 4]

print("=" * 50)
print("입력 데이터:")
print(f"  States:  {states}")
print(f"  Actions: {actions}")
print(f"  Rewards: {rewards}")
print("=" * 50)
print()

sequence = create_dt_sequence_verbose(states, actions, rewards)

In [ ]:
def create_dt_sequence(states, actions, rewards):
    """
    Decision Transformer 시퀀스 생성
    """
    rtgs = calculate_rtg(rewards)
    
    sequence = []
    for rtg, state, action in zip(rtgs, states, actions):
        sequence.append(('RTG', rtg))
        sequence.append(('State', state))
        sequence.append(('Action', action))
    
    return sequence

# 예시 데이터
states = ['s0', 's1', 's2', 's3']
actions = ['a0', 'a1', 'a2', 'a3']
rewards = [1, 2, 3, 4]

sequence = create_dt_sequence(states, actions, rewards)

print("Decision Transformer 시퀀스:")
print("="*60)
for i, (token_type, value) in enumerate(sequence):
    print(f"Position {i:2d}: [{token_type:6s}] = {value}")

In [ ]:
# 인덱싱 규칙을 코드로 확인

sequence = ['R0', 's0', 'a0', 'R1', 's1', 'a1', 'R2', 's2', 'a2', 'R3', 's3', 'a3']

print("전체 시퀀스:")
for i, token in enumerate(sequence):
    print(f"  Position {i:2d}: {token}")

print()
print("=" * 50)
print("Python 슬라이싱으로 각 타입 추출:")
print("=" * 50)

# RTG 위치 (0, 3, 6, 9, ...)
rtg_positions = sequence[0::3]
print(f"\nRTG (0::3):    {rtg_positions}")
print(f"  → 위치: {list(range(0, len(sequence), 3))}")

# State 위치 (1, 4, 7, 10, ...)
state_positions = sequence[1::3]
print(f"\nState (1::3):  {state_positions}")
print(f"  → 위치: {list(range(1, len(sequence), 3))}")
print(f"  → Action 예측은 이 위치에서!")

# Action 위치 (2, 5, 8, 11, ...)
action_positions = sequence[2::3]
print(f"\nAction (2::3): {action_positions}")
print(f"  → 위치: {list(range(2, len(sequence), 3))}")

print()
print("┌───────────────────────────────────────────────────┐")
print("│  0::3 의미: 0번부터 시작, 3칸씩 점프              │")
print("│  1::3 의미: 1번부터 시작, 3칸씩 점프              │")
print("│  2::3 의미: 2번부터 시작, 3칸씩 점프              │")
print("└───────────────────────────────────────────────────┘")

In [ ]:
# Causal Masking과 함께 이해하기
# 각 위치에서 어떤 정보를 볼 수 있는지 확인

sequence = ['R0', 's0', 'a0', 'R1', 's1', 'a1', 'R2', 's2', 'a2']

print("각 State 위치에서 볼 수 있는 정보 (Causal Masking):")
print("=" * 60)

# State 위치: 1, 4, 7
state_indices = [1, 4, 7]

for idx in state_indices:
    visible = sequence[:idx+1]  # 자기 자신까지 볼 수 있음
    action_to_predict = sequence[idx+1] if idx+1 < len(sequence) else "?"
    
    print(f"\n위치 {idx} ({sequence[idx]})에서:")
    print(f"  볼 수 있는 토큰: {visible}")
    print(f"  볼 수 없는 토큰: {sequence[idx+1:]}")
    print(f"  → {visible[-2]}(RTG)와 {visible[-1]}(State)를 보고")
    print(f"  → {action_to_predict}를 예측!")

print()
print("┌───────────────────────────────────────────────────────┐")
print("│  핵심: State 위치에서는 해당 시점의 RTG와 State만    │")
print("│        볼 수 있기 때문에, 이 정보로 Action을 예측     │")
print("└───────────────────────────────────────────────────────┘")

In [ ]:
# 시퀀스 구조 및 예측 위치 시각화

# colors 정의 (시각화용)
colors = {'RTG': '#ff6b6b', 'State': '#4ecdc4', 'Action': '#45b7d1'}

# 시퀀스 생성 (cell-11에서 정의된 함수 사용)
def create_dt_sequence(states, actions, rewards):
    rtgs = calculate_rtg(rewards)
    sequence = []
    for rtg, state, action in zip(rtgs, states, actions):
        sequence.append(('RTG', rtg))
        sequence.append(('State', state))
        sequence.append(('Action', action))
    return sequence

states = ['s0', 's1', 's2', 's3']
actions = ['a0', 'a1', 'a2', 'a3']
rewards = [1, 2, 3, 4]
sequence = create_dt_sequence(states, actions, rewards)

# 시각화
fig, ax = plt.subplots(figsize=(14, 4))

# 입력 시퀀스
for i, (token_type, value) in enumerate(sequence):
    color = colors[token_type]
    rect = plt.Rectangle((i, 1), 0.9, 1, facecolor=color, edgecolor='black', alpha=0.7)
    ax.add_patch(rect)
    ax.text(i + 0.45, 1.5, str(value), ha='center', va='center', fontsize=9)

# 예측 화살표 (State 위치에서)
state_positions = [i for i, (t, _) in enumerate(sequence) if t == 'State']
for i, pos in enumerate(state_positions):
    ax.annotate('', xy=(pos + 0.45, 0.7), xytext=(pos + 0.45, 0.9),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))
    ax.text(pos + 0.45, 0.4, f'→a{i}', ha='center', va='center', 
            fontsize=11, fontweight='bold', color='red')

ax.set_xlim(-0.5, len(sequence) + 0.5)
ax.set_ylim(0, 2.5)
ax.set_xticks(range(len(sequence)))
ax.set_xlabel('Position')
ax.set_yticks([0.5, 1.5])
ax.set_yticklabels(['Prediction', 'Input'])
ax.set_title('State 위치(1::3)에서 Action 예측')

plt.tight_layout()
plt.show()

print(f"State 위치: {state_positions} (인덱스 1::3)")
print(f"이 위치들에서 다음 Action을 예측합니다.")

### 2.3 Causal Masking과 함께 이해하기

Causal masking 덕분에 각 위치에서 볼 수 있는 정보가 제한됩니다.

In [ ]:
# 각 State 위치에서 볼 수 있는 정보
print("각 State 위치에서 볼 수 있는 정보 (Causal Masking):")
print("="*60)

for t, pos in enumerate(state_positions):
    visible = sequence[:pos+1]  # 자기 자신까지 볼 수 있음
    visible_str = [f"{v}" for _, v in visible]
    print(f"\ns{t} 위치 (pos={pos})에서 a{t} 예측:")
    print(f"  볼 수 있는 정보: {visible_str}")
    print(f"  → R{t}와 s{t}를 보고 a{t}를 예측!")

### 3.1 기존 RL: Bellman Equation 기반

기존 강화학습의 핵심 아이디어:

```
"최적의 행동 = 미래 보상의 기대값을 최대화하는 행동"
```

**Q-Learning** (Value-based):
$$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$

**Policy Gradient** (Policy-based):
$$\nabla J(\theta) = \mathbb{E}[\nabla \log \pi_\theta(a|s) \cdot A(s, a)]$$

핵심은 **Value Function** (Q 또는 V)을 추정한다는 것!

In [ ]:
# Decision Transformer 접근법 (의사 코드)

class SimpleDTConcept:
    """
    Decision Transformer 개념 설명용 클래스
    
    핵심 차이:
    - Q-table/Value function 없음!
    - 그냥 입력→출력 매핑을 학습
    """
    def __init__(self):
        # Transformer 모델 (여기서는 개념만)
        self.model = "Transformer(RTG, State → Action)"
    
    def train(self, dataset):
        """
        학습: 과거 데이터에서 패턴 학습
        
        데이터셋: [(rtg, state, action), ...]
        손실: MSE(예측 action, 실제 action)
        
        Q-Learning과 달리:
        - TD error 계산 없음
        - Bellman equation 없음
        - 그냥 Supervised Learning!
        """
        for rtg, state, action in dataset:
            predicted_action = self.model(rtg, state)
            loss = (predicted_action - action) ** 2  # MSE
            # loss.backward() & optimizer.step()
    
    def select_action(self, state, target_return):
        """
        추론: 목표 return과 state를 주면 action 출력
        
        Q-Learning: argmax_a Q(s, a)  ← Q값 계산 필요
        DT:         model(target_return, state)  ← 그냥 forward pass
        """
        return self.model(target_return, state)

# 두 접근법 비교
print("=" * 60)
print("두 접근법 비교")
print("=" * 60)

comparison_steps = """
┌──────────────────────────────────────────────────────────────┐
│                    Q-Learning                                │
├──────────────────────────────────────────────────────────────┤
│  1. 환경과 상호작용하며 경험 수집                            │
│  2. Q(s,a) 값을 Bellman equation으로 업데이트                │
│  3. 추론: argmax_a Q(s,a) 로 행동 선택                       │
│                                                              │
│  필요한 것: Q-table 또는 Q-network, TD error 계산            │
└──────────────────────────────────────────────────────────────┘

┌──────────────────────────────────────────────────────────────┐
│                Decision Transformer                          │
├──────────────────────────────────────────────────────────────┤
│  1. 과거 데이터(궤적) 수집 (Offline)                         │
│  2. (RTG, State) → Action 매핑을 학습                        │
│  3. 추론: model(target_RTG, state) 로 행동 출력             │
│                                                              │
│  필요한 것: Transformer, MSE loss                            │
│  필요 없는 것: Value function, Bellman equation              │
└──────────────────────────────────────────────────────────────┘
"""
print(comparison_steps)

### 3.3 장단점 비교

**Decision Transformer의 장점:**

```
1. 단순함
   - Supervised learning으로 귀결
   - 구현이 쉽고 디버깅이 쉬움

2. 안정성
   - Value function 추정의 불안정성 없음
   - Q-learning의 overestimation 문제 없음

3. 목표 지향적
   - 원하는 return을 직접 지정 가능
   - "100점 짜리 플레이" vs "50점 짜리 플레이" 선택 가능

4. 장기 의존성
   - Transformer가 긴 시퀀스 처리에 강함
   - 과거 많은 스텝의 정보 활용 가능
```

**단점/한계:**

```
1. Offline 데이터 의존
   - 학습 데이터에 없는 행동은 못 배움
   - 데이터 품질이 성능을 결정

2. 목표 설정의 어려움
   - 적절한 target RTG 설정이 필요
   - 너무 높으면 이상한 행동, 너무 낮으면 성능 저하

3. 계산 비용
   - Transformer는 연산량이 많음
   - 실시간 적용에 제약
```

### 3.1 기존 RL: Bellman Equation

**Q-Learning**:
$$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$

**Policy Gradient**:
$$\nabla J(\theta) = \mathbb{E}[\nabla \log \pi_\theta(a|s) \cdot A(s, a)]$$

### 3.2 Decision Transformer: Sequence Modeling

**핵심 아이디어**:
- Bellman equation 없음
- Value function 없음
- 단순히 **시퀀스 예측 문제**로 변환

$$\pi(a | s, R̂) = \text{Transformer}(R̂_1, s_1, a_1, ..., R̂_t, s_t)$$

In [ ]:
# 비교 표
comparison = """
┌─────────────────────┬────────────────────────┬────────────────────────┐
│        항목         │       기존 RL          │  Decision Transformer  │
├─────────────────────┼────────────────────────┼────────────────────────┤
│ 학습 목표           │ Value function 추정    │ 다음 Action 예측       │
├─────────────────────┼────────────────────────┼────────────────────────┤
│ 손실 함수           │ TD error, PG loss      │ Cross-entropy / MSE    │
├─────────────────────┼────────────────────────┼────────────────────────┤
│ 조건화              │ State만                │ State + RTG (목표)     │
├─────────────────────┼────────────────────────┼────────────────────────┤
│ 시간적 의존성       │ 1-step (Markov)        │ 전체 컨텍스트 (K steps)│
├─────────────────────┼────────────────────────┼────────────────────────┤
│ 아키텍처            │ MLP, CNN               │ Transformer            │
├─────────────────────┼────────────────────────┼────────────────────────┤
│ 데이터 활용         │ Online / Offline       │ Offline                │
└─────────────────────┴────────────────────────┴────────────────────────┘
"""
print(comparison)

In [ ]:
class SimpleEmbeddings(nn.Module):
    """
    Decision Transformer의 임베딩 레이어 (간단 버전)
    
    역할: RTG, State, Action을 동일한 차원의 벡터로 변환
    """
    def __init__(self, state_dim, action_dim, hidden_size, max_timestep=1000):
        super().__init__()
        
        # ┌─────────────────────────────────────────────────────┐
        # │  각 입력 타입별 임베딩 레이어                       │
        # │                                                     │
        # │  RTG:    스칼라(1) → hidden_size 벡터              │
        # │  State:  state_dim → hidden_size 벡터              │
        # │  Action: action_dim → hidden_size 벡터             │
        # │                                                     │
        # │  모두 같은 hidden_size로 변환해야 시퀀스 구성 가능! │
        # └─────────────────────────────────────────────────────┘
        self.embed_state = nn.Linear(state_dim, hidden_size)
        self.embed_action = nn.Linear(action_dim, hidden_size)
        self.embed_return = nn.Linear(1, hidden_size)  # RTG는 스칼라
        
        # Timestep 임베딩 (Phase 1에서 배운 Learned Positional Embedding!)
        self.embed_timestep = nn.Embedding(max_timestep, hidden_size)
        
        # Layer normalization (학습 안정화)
        self.embed_ln = nn.LayerNorm(hidden_size)
        
    def forward(self, states, actions, returns_to_go, timesteps):
        """
        Args:
            states: (batch, seq_len, state_dim)
            actions: (batch, seq_len, action_dim)
            returns_to_go: (batch, seq_len, 1)
            timesteps: (batch, seq_len)
        Returns:
            token_embeddings: (batch, seq_len*3, hidden_size)
        """
        batch_size, seq_len = states.shape[0], states.shape[1]
        
        # Step 1: 각 입력을 hidden_size 차원으로 변환
        state_emb = self.embed_state(states)      # (B, T, H)
        action_emb = self.embed_action(actions)   # (B, T, H)
        return_emb = self.embed_return(returns_to_go)  # (B, T, H)
        
        # Step 2: Timestep 임베딩 추가 (위치 정보)
        time_emb = self.embed_timestep(timesteps)  # (B, T, H)
        
        state_emb = state_emb + time_emb
        action_emb = action_emb + time_emb
        return_emb = return_emb + time_emb
        
        # Step 3: 시퀀스 인터리빙
        # [R₀, s₀, a₀, R₁, s₁, a₁, ...] 순서로 배치
        # (B, T, 3, H) → (B, T*3, H)
        stacked = torch.stack([return_emb, state_emb, action_emb], dim=2)
        token_embeddings = stacked.reshape(batch_size, seq_len * 3, -1)
        
        return self.embed_ln(token_embeddings)

# 테스트: 임베딩 레이어 동작 확인
print("=" * 60)
print("임베딩 레이어 테스트")
print("=" * 60)

embedder = SimpleEmbeddings(
    state_dim=4,      # 상태 벡터 차원 (예: CartPole)
    action_dim=2,     # 행동 벡터 차원 (예: 좌/우)
    hidden_size=32    # 임베딩 차원
)

# 임베딩 레이어 파라미터 확인
print("\n임베딩 레이어 구조:")
for name, param in embedder.named_parameters():
    print(f"  {name}: {param.shape}")

### 4.1 임베딩 레이어

In [ ]:
class SimpleEmbeddings(nn.Module):
    """
    Decision Transformer의 임베딩 레이어 (간단 버전)
    """
    def __init__(self, state_dim, action_dim, hidden_size, max_timestep=1000):
        super().__init__()
        
        # 각 입력 타입별 임베딩
        self.embed_state = nn.Linear(state_dim, hidden_size)
        self.embed_action = nn.Linear(action_dim, hidden_size)
        self.embed_return = nn.Linear(1, hidden_size)
        
        # Timestep 임베딩
        self.embed_timestep = nn.Embedding(max_timestep, hidden_size)
        
        # Layer normalization
        self.embed_ln = nn.LayerNorm(hidden_size)
        
    def forward(self, states, actions, returns_to_go, timesteps):
        """
        Args:
            states: (batch, seq_len, state_dim)
            actions: (batch, seq_len, action_dim)
            returns_to_go: (batch, seq_len, 1)
            timesteps: (batch, seq_len)
        Returns:
            token_embeddings: (batch, seq_len*3, hidden_size)
        """
        batch_size, seq_len = states.shape[0], states.shape[1]
        
        # 각 입력 임베딩
        state_emb = self.embed_state(states)      # (B, T, H)
        action_emb = self.embed_action(actions)   # (B, T, H)
        return_emb = self.embed_return(returns_to_go)  # (B, T, H)
        
        # Timestep 임베딩 추가
        time_emb = self.embed_timestep(timesteps)  # (B, T, H)
        
        state_emb = state_emb + time_emb
        action_emb = action_emb + time_emb
        return_emb = return_emb + time_emb
        
        # 시퀀스 인터리빙: [R, s, a, R, s, a, ...]
        # (B, T, 3, H) → (B, T*3, H)
        stacked = torch.stack([return_emb, state_emb, action_emb], dim=2)
        token_embeddings = stacked.reshape(batch_size, seq_len * 3, -1)
        
        return self.embed_ln(token_embeddings)

# 테스트
embedder = SimpleEmbeddings(
    state_dim=4,
    action_dim=2,
    hidden_size=32
)

# 더미 데이터
batch_size, seq_len = 2, 5
states = torch.randn(batch_size, seq_len, 4)
actions = torch.randn(batch_size, seq_len, 2)
rtgs = torch.randn(batch_size, seq_len, 1)
timesteps = torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1)

embeddings = embedder(states, actions, rtgs, timesteps)

print(f"입력:")
print(f"  States: {states.shape}")
print(f"  Actions: {actions.shape}")
print(f"  RTGs: {rtgs.shape}")
print(f"  Timesteps: {timesteps.shape}")
print(f"\n출력 (인터리빙된 시퀀스):")
print(f"  Embeddings: {embeddings.shape}")
print(f"  (batch=2, seq_len=5*3=15, hidden=32)")

### 4.2 Action 예측 추출

In [ ]:
def extract_action_predictions(transformer_output, seq_len):
    """
    Transformer 출력에서 Action 예측만 추출
    
    시퀀스: [R₀, s₀, a₀, R₁, s₁, a₁, ...]
    위치:    0   1   2   3   4   5
    
    Action 예측은 State 위치 (1, 4, 7, ... = 1::3)에서 추출
    """
    # State 위치에서 예측 추출
    action_preds = transformer_output[:, 1::3, :]  # (B, T, H)
    return action_preds

# 테스트
# 가상의 transformer 출력
fake_output = torch.randn(2, 15, 32)  # (batch=2, seq=15, hidden=32)

action_preds = extract_action_predictions(fake_output, seq_len=5)

print(f"Transformer 출력: {fake_output.shape}")
print(f"Action 예측 (1::3 위치): {action_preds.shape}")
print(f"\n추출된 위치: {list(range(1, 15, 3))}")

### 4.3 간단한 Decision Transformer 모델

아래는 `SimpleDecisionTransformer`의 전체 아키텍처입니다:

```mermaid
flowchart TB
    subgraph Inputs["📥 입력"]
        RTG["RTG<br/>(batch, seq, 1)"]
        State["State<br/>(batch, seq, state_dim)"]
        Action["Action<br/>(batch, seq, action_dim)"]
        Time["Timestep<br/>(batch, seq)"]
    end

    subgraph Embedding["1️⃣ 임베딩 레이어"]
        direction TB
        RTG --> |"Linear(1→H)"| RTG_emb["RTG Embedding<br/>(batch, seq, H)"]
        State --> |"Linear(state_dim→H)"| State_emb["State Embedding<br/>(batch, seq, H)"]
        Action --> |"Linear(action_dim→H)"| Action_emb["Action Embedding<br/>(batch, seq, H)"]
        Time --> |"Embedding(max_t, H)"| Time_emb["Time Embedding<br/>(batch, seq, H)"]
        
        RTG_emb --> |"+"| RTG_final["R + T"]
        Time_emb --> RTG_final
        State_emb --> |"+"| State_final["S + T"]
        Time_emb --> State_final
        Action_emb --> |"+"| Action_final["A + T"]
        Time_emb --> Action_final
    end

    subgraph Interleave["2️⃣ 시퀀스 인터리빙"]
        RTG_final --> Stack
        State_final --> Stack
        Action_final --> Stack
        Stack["torch.stack & reshape"] --> Sequence["[R₀,s₀,a₀,R₁,s₁,a₁,...]<br/>(batch, seq×3, H)"]
        Sequence --> LN["LayerNorm"]
    end

    subgraph Transformer["3️⃣ Transformer Encoder"]
        LN --> Mask["Causal Mask 적용<br/>(미래 토큰 차단)"]
        Mask --> TF["TransformerEncoder<br/>× n_layers"]
        TF --> Output["(batch, seq×3, H)"]
    end

    subgraph Prediction["4️⃣ Action 예측"]
        Output --> Extract["State 위치 추출<br/>[:, 1::3, :]"]
        Extract --> Head["Linear(H→action_dim)"]
        Head --> ActionPred["Action Predictions<br/>(batch, seq, action_dim)"]
    end

    style Inputs fill:#e1f5fe
    style Embedding fill:#fff3e0
    style Interleave fill:#f3e5f5
    style Transformer fill:#e8f5e9
    style Prediction fill:#ffebee
```

**핵심 구성 요소:**

| 단계 | 레이어 | 역할 |
|:---:|:---|:---|
| 1️⃣ | `embed_*` + `embed_timestep` | 각 입력을 hidden_size 차원으로 변환 + 시간 정보 추가 |
| 2️⃣ | `torch.stack` + `reshape` | (R,s,a) 트리플을 인터리빙하여 시퀀스 구성 |
| 3️⃣ | `TransformerEncoder` | Causal Attention으로 시퀀스 처리 |
| 4️⃣ | `predict_action` | State 위치(1::3)에서 Action 예측 |

In [ ]:
class SimpleDecisionTransformer(nn.Module):
    """
    간단한 Decision Transformer 모델
    
    핵심 구조:
    1. 임베딩: RTG, State, Action → 동일 차원 벡터
    2. Transformer: 시퀀스 처리 (Causal Masking)
    3. 예측 헤드: State 위치에서 Action 예측
    """
    def __init__(self, state_dim, action_dim, hidden_size=64, n_heads=4, n_layers=2, max_timestep=1000):
        super().__init__()
        
        self.hidden_size = hidden_size
        
        # ┌─────────────────────────────────────────────────────┐
        # │  1. 임베딩 레이어                                   │
        # └─────────────────────────────────────────────────────┘
        self.embed_state = nn.Linear(state_dim, hidden_size)
        self.embed_action = nn.Linear(action_dim, hidden_size)
        self.embed_return = nn.Linear(1, hidden_size)
        self.embed_timestep = nn.Embedding(max_timestep, hidden_size)
        self.embed_ln = nn.LayerNorm(hidden_size)
        
        # ┌─────────────────────────────────────────────────────┐
        # │  2. Transformer (Causal Self-Attention)             │
        # └─────────────────────────────────────────────────────┘
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=n_heads,
            dim_feedforward=hidden_size * 4,
            dropout=0.1,
            activation='gelu',
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        # ┌─────────────────────────────────────────────────────┐
        # │  3. 예측 헤드 (Action 예측)                         │
        # └─────────────────────────────────────────────────────┘
        self.predict_action = nn.Linear(hidden_size, action_dim)
    
    def forward(self, states, actions, returns_to_go, timesteps):
        """
        Args:
            states: (batch, seq_len, state_dim)
            actions: (batch, seq_len, action_dim)
            returns_to_go: (batch, seq_len, 1)
            timesteps: (batch, seq_len)
        Returns:
            action_preds: (batch, seq_len, action_dim)
        """
        batch_size, seq_len = states.shape[0], states.shape[1]
        
        # Step 1: 임베딩
        state_emb = self.embed_state(states)
        action_emb = self.embed_action(actions)
        return_emb = self.embed_return(returns_to_go)
        time_emb = self.embed_timestep(timesteps)
        
        # Timestep 임베딩 추가
        state_emb = state_emb + time_emb
        action_emb = action_emb + time_emb
        return_emb = return_emb + time_emb
        
        # Step 2: 시퀀스 인터리빙 [R, s, a, R, s, a, ...]
        stacked = torch.stack([return_emb, state_emb, action_emb], dim=2)
        sequence = stacked.reshape(batch_size, seq_len * 3, self.hidden_size)
        sequence = self.embed_ln(sequence)
        
        # Step 3: Causal Mask 생성
        # 각 토큰은 자기 자신과 이전 토큰만 볼 수 있음
        seq_length = seq_len * 3
        causal_mask = torch.triu(
            torch.ones(seq_length, seq_length, device=sequence.device),
            diagonal=1
        ).bool()
        
        # Step 4: Transformer 통과
        output = self.transformer(sequence, mask=causal_mask)
        
        # Step 5: State 위치(1::3)에서 Action 예측 추출
        state_outputs = output[:, 1::3, :]  # (batch, seq_len, hidden)
        action_preds = self.predict_action(state_outputs)
        
        return action_preds

# 모델 생성 및 테스트
print("=" * 60)
print("Simple Decision Transformer 테스트")
print("=" * 60)

model = SimpleDecisionTransformer(
    state_dim=4,      # 예: CartPole 상태 차원
    action_dim=2,     # 예: 좌/우 행동
    hidden_size=32,
    n_heads=2,
    n_layers=2
)

# 더미 데이터
batch_size, seq_len = 2, 5
states = torch.randn(batch_size, seq_len, 4)
actions = torch.randn(batch_size, seq_len, 2)
rtgs = torch.randn(batch_size, seq_len, 1)
timesteps = torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1)

# Forward pass
action_preds = model(states, actions, rtgs, timesteps)

print(f"\n입력:")
print(f"  States:    {states.shape}")
print(f"  Actions:   {actions.shape}")
print(f"  RTGs:      {rtgs.shape}")
print(f"  Timesteps: {timesteps.shape}")
print(f"\n출력:")
print(f"  Action predictions: {action_preds.shape}")
print(f"\n모델 파라미터 수: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
def train_step(model, optimizer, states, actions, rtgs, timesteps):
    """
    Decision Transformer 학습 스텝 (의사 코드)
    
    핵심: Supervised Learning과 동일!
    - 입력: (RTG, State, Action) 시퀀스
    - 출력: Action 예측
    - 손실: 예측 Action vs 실제 Action (MSE)
    """
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    action_preds = model(states, actions, rtgs, timesteps)
    
    # Loss 계산 (MSE: 연속 행동 공간)
    # 또는 CrossEntropy: 이산 행동 공간
    loss = nn.MSELoss()(action_preds, actions)
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    return loss.item()

# 학습 루프 예시
print("=" * 60)
print("학습 루프 예시")
print("=" * 60)

# 모델과 옵티마이저
model = SimpleDecisionTransformer(state_dim=4, action_dim=2, hidden_size=32)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# 더미 학습 데이터
batch_size, seq_len = 8, 10
dummy_data = {
    'states': torch.randn(batch_size, seq_len, 4),
    'actions': torch.randn(batch_size, seq_len, 2),
    'rtgs': torch.randn(batch_size, seq_len, 1),
    'timesteps': torch.arange(seq_len).unsqueeze(0).expand(batch_size, -1)
}

# 간단한 학습 루프
print("\n학습 진행:")
for epoch in range(5):
    loss = train_step(
        model, optimizer,
        dummy_data['states'],
        dummy_data['actions'],
        dummy_data['rtgs'],
        dummy_data['timesteps']
    )
    print(f"  Epoch {epoch+1}: Loss = {loss:.4f}")

print()
print("┌─────────────────────────────────────────────────────────┐")
print("│  핵심 포인트:                                           │")
print("│  1. Bellman equation 없음 - 그냥 MSE Loss!              │")
print("│  2. TD error 계산 없음 - Supervised Learning!           │")
print("│  3. Value function 없음 - 직접 Action 예측!             │")
print("└─────────────────────────────────────────────────────────┘")

### 4.4 학습 루프 스케치

---
## 연습 문제

### 연습 1: RTG 계산

다음 보상 시퀀스에 대해 RTG를 계산하세요.

In [ ]:
rewards = [10, -5, 20, 15, -3, 8]

# TODO: RTG 계산
rtg = []

# 정답 확인
# print(f"RTG: {rtg}")
# 예상: [45, 35, 40, 20, 5, 8]

<details>
<summary>정답 보기</summary>

```python
rtg = calculate_rtg(rewards)
# 또는
rtg = [sum(rewards[i:]) for i in range(len(rewards))]
print(f"RTG: {rtg}")  # [45, 35, 40, 20, 5, 8]
```
</details>

### 연습 2: 시퀀스 인덱싱

시퀀스 `[R0, s0, a0, R1, s1, a1, R2, s2, a2, R3, s3, a3]`에서:

1. RTG 위치는?
2. State 위치는?
3. Action 위치는?

In [ ]:
sequence = ['R0', 's0', 'a0', 'R1', 's1', 'a1', 'R2', 's2', 'a2', 'R3', 's3', 'a3']

# TODO: 각 타입의 위치 찾기
# rtg_positions = sequence[?::?]
# state_positions = sequence[?::?]
# action_positions = sequence[?::?]

<details>
<summary>정답 보기</summary>

```python
rtg_positions = sequence[0::3]    # ['R0', 'R1', 'R2', 'R3']
state_positions = sequence[1::3]  # ['s0', 's1', 's2', 's3']
action_positions = sequence[2::3] # ['a0', 'a1', 'a2', 'a3']

print(f"RTG 위치 (0::3): {rtg_positions}")
print(f"State 위치 (1::3): {state_positions}")
print(f"Action 위치 (2::3): {action_positions}")
```
</details>

### 연습 3: 추론 시뮬레이션

목표 return이 50이고, 매 스텝 reward가 `[15, 20, 10]`일 때,
각 스텝에서의 RTG를 계산하세요.

In [ ]:
target_return = 50
rewards = [15, 20, 10]

# TODO: 각 스텝에서의 RTG 계산
# rtg_step0 = ?
# rtg_step1 = ?
# rtg_step2 = ?
# rtg_step3 = ?

<details>
<summary>정답 보기</summary>

```python
rtg = target_return
print(f"Step 0 시작: RTG = {rtg}")  # 50

for i, r in enumerate(rewards):
    rtg = rtg - r
    print(f"Step {i} 후 (reward={r}): RTG = {rtg}")

# Step 0 시작: RTG = 50
# Step 0 후 (reward=15): RTG = 35
# Step 1 후 (reward=20): RTG = 15
# Step 2 후 (reward=10): RTG = 5
```
</details>

---

# 🎉 Phase 2 완료!

## 📚 배운 내용 요약

### 🎯 Return-to-Go (RTG)
| 개념 | 설명 |
|:---:|:---|
| **정의** | 현재 시점부터 끝까지의 보상 합 |
| **학습 시** | 과거 데이터에서 계산된 실제 RTG 사용 |
| **추론 시** | 원하는 목표를 RTG로 설정 |
| **업데이트** | 새 RTG = 이전 RTG - 받은 보상 |

### 📦 시퀀스 구성
| 개념 | 설명 |
|:---:|:---|
| **구조** | [R₀, s₀, a₀, R₁, s₁, a₁, ...] |
| **RTG 위치** | 0::3 (0, 3, 6, ...) |
| **State 위치** | 1::3 (1, 4, 7, ...) ← **Action 예측 위치** |
| **Action 위치** | 2::3 (2, 5, 8, ...) |

### 🔄 기존 RL과의 차이
| 기존 RL | Decision Transformer |
|:---:|:---:|
| Value Function 추정 | 시퀀스 예측 |
| Bellman Equation | Supervised Learning |
| State → Action | (RTG, State) → Action |

---

## ➡️ 다음 단계

**Phase 3: Gym 환경 실습** → `phase3_gym_practice.ipynb`

- 실제 Gym 환경에서 DT 실행
- 데이터 수집 및 전처리
- 학습 및 평가

---

## ✅ 체크리스트

다음으로 넘어가기 전에 확인하세요:

- [ ] RTG가 무엇인지 설명할 수 있다
- [ ] RTG를 직접 계산할 수 있다
- [ ] 학습 vs 추론에서 RTG의 역할 차이를 이해했다
- [ ] (RTG, State, Action) 시퀀스 구조를 이해했다
- [ ] 왜 State 위치에서 Action을 예측하는지 이해했다
- [ ] DT와 기존 RL의 핵심 차이를 설명할 수 있다